<a href="https://colab.research.google.com/github/mahima8178/100-pandas-puzzles/blob/master/new-aug-with-models-comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/My Drive/Kaggle"

In [3]:
%cd /content/drive/MyDrive/Kaggle/SPECT IMG/

/content/drive/MyDrive/Kaggle/SPECT IMG


In [8]:
!pip install opencv-python==4.5.2.54
!pip install --upgrade albumentations

     |████████████████████████████████| 102kB 7.0MB/s 
     |████████████████████████████████| 38.2MB 79kB/s 
  Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [5]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
print('')
print('Detailed information:')
print('---------------------')
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

Found GPU at: /device:GPU:0

Detailed information:
---------------------
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13051266789155382024
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14509932544
locality {
  bus_id: 1
  links {
  }
}
incarnation: 12089584587824537886
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
]


In [6]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.layers import Input,Dense,GlobalAveragePooling2D,Flatten,concatenate,BatchNormalization, Dropout
from tensorflow.keras.applications import InceptionV3,DenseNet121
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K

# Visualize the Train/Val loss
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
!pip install git+https://github.com/mjkvaak/ImageDataAugmentor

  Cloning https://github.com/mjkvaak/ImageDataAugmentor to /tmp/pip-req-build-reaeo45g
  Running command git clone -q https://github.com/mjkvaak/ImageDataAugmentor /tmp/pip-req-build-reaeo45g
     |████████████████████████████████| 51.0MB 59kB/s 
  Created wheel for ImageDataAugmentor: filename=ImageDataAugmentor-0.0.0-cp37-none-any.whl size=29545 sha256=70afa507ceae3fbfd1d9fc9b50e88a4ca9500f747ed70489f27cbd4ca1e99385
  Stored in directory: /tmp/pip-ephem-wheel-cache-5nq2uw8d/wheels/d9/10/55/6fca35a4072f87d694876d56ece64db3846cf45e1da1c381fe
Successfully built ImageDataAugmentor
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30


In [9]:
import tensorflow as tf
from ImageDataAugmentor.image_data_augmentor import *
import albumentations

In [10]:
def get_model(model):
# Load the pretained model
    kwargs =    {'input_shape':(224, 224, 3),
                'include_top':False,
                'weights':'imagenet',
                'pooling':'avg'}
    
    pretrained_model = model(**kwargs)
    pretrained_model.trainable = False
    
    inputs = pretrained_model.input

    x = tf.keras.layers.Dense(1024, activation='relu')(pretrained_model.output)
    x = tf.keras.layers.Dense(256, activation='relu')(x)

    outputs = tf.keras.layers.Dense(2, activation='softmax')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model

In [12]:

AUGMENTATIONS = albumentations.Compose([
    albumentations.OneOf([
        albumentations.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3),
        albumentations.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1)
    ],p=1),
    albumentations.RandomRotate90(p=1),
    albumentations.Rotate(p=1),
    albumentations.Posterize(p=1),
    albumentations.RandomSnow(p=1),
    albumentations.VerticalFlip(p=0.8)
])

In [16]:
shear_range = 0.1 #@param {type:"slider", min:0.0, max:1.0, step:0.05}
zoom_range = 0.1 #@param {type:"slider", min:0.0, max:1.0, step:0.05}
width_shift_range = 0.1 #@param {type:"slider", min:0.0, max:1.0, step:0.05}
height_shift_range = 0.1 #@param {type:"slider", min:0.0, max:1.0, step:0.05}
rotation_range = 10 #@param {type:"slider", min:0, max:90, step:5}
horizontal_flip = True #@param {type:"boolean"}
vertical_flip = False #@param {type:"boolean"}
#@markdown Data source (No need to change if the download succeeded.)
data_directory = ''
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
        validation_split=0.2
    )
train_datagen = ImageDataAugmentor(
        rescale=1./255,
        augment=AUGMENTATIONS)

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input)

val_datagen   = ImageDataGenerator(rescale=1./255) 
train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=shear_range,
    zoom_range=zoom_range,
    width_shift_range=width_shift_range,
    height_shift_range=height_shift_range,
    rotation_range=rotation_range,
    horizontal_flip=horizontal_flip,
    vertical_flip=vertical_flip) 

train_generator = train_datagen.flow_from_directory(os.path.join(data_directory, 'TRAIN/'), # this is where you specify the path to the main data folder
                                                 target_size=(90,90),
                                                 batch_size=64,
                                                 class_mode='categorical',
                                                 shuffle=True)

    # Data Generator for validation without data augmentation!

val_generator = val_datagen.flow_from_directory(os.path.join(data_directory, 'VAL/'), # this is where you specify the path to the main data folder
                                                 target_size=(90,90),
                                                 batch_size=64,
                                                 class_mode='categorical',
                                                 shuffle=True)

Found 1691 images belonging to 2 classes.
Found 345 images belonging to 2 classes.


In [18]:
def create_gen():
    # Load the Images with a generator and Data Augmentation
    shear_range = 0.1 
    zoom_range = 0.1 
    width_shift_range = 0.1 
    height_shift_range = 0.1 
    rotation_range = 10 
    horizontal_flip = True 
    vertical_flip = False 
    data_directory = ''
    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
        validation_split=0.2
    )
    train_datagen = ImageDataAugmentor(
        rescale=1./255,
        augment=AUGMENTATIONS)

    val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
    )

    val_datagen   = ImageDataGenerator(rescale=1./255) 
    train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=shear_range,
    zoom_range=zoom_range,
    width_shift_range=width_shift_range,
    height_shift_range=height_shift_range,
    rotation_range=rotation_range,
    horizontal_flip=horizontal_flip,
    vertical_flip=vertical_flip) 

    train_generator = train_datagen.flow_from_directory(os.path.join(data_directory, 'TRAIN/'), # this is where you specify the path to the main data folder
                                                 target_size=(90,90),
                                                 batch_size=64,
                                                 class_mode='categorical',
                                                 shuffle=True)

    # Data Generator for validation without data augmentation!

    val_generator = val_datagen.flow_from_directory(os.path.join(data_directory, 'VAL/'), # this is where you specify the path to the main data folder
                                                 target_size=(90,90),
                                                 batch_size=64,
                                                 class_mode='categorical',
                                                 shuffle=True)
    
    return train_datagen,val_datagen,train_generator,val_generator

In [19]:
from time import perf_counter

In [ ]:
# Dictionary with the models
models = {
    "DenseNet121": {"model":tf.keras.applications.DenseNet121, "perf":0},
    "MobileNetV2": {"model":tf.keras.applications.MobileNetV2, "perf":0},
    "DenseNet169": {"model":tf.keras.applications.DenseNet169, "perf":0},
    "DenseNet201": {"model":tf.keras.applications.DenseNet201, "perf":0},
    "InceptionResNetV2": {"model":tf.keras.applications.InceptionResNetV2, "perf":0},
    "InceptionV3": {"model":tf.keras.applications.InceptionV3, "perf":0},
    "MobileNet": {"model":tf.keras.applications.MobileNet, "perf":0},
    "MobileNetV2": {"model":tf.keras.applications.MobileNetV2, "perf":0},
    "MobileNetV3Large": {"model":tf.keras.applications.MobileNetV3Large, "perf":0},
    "MobileNetV3Small": {"model":tf.keras.applications.MobileNetV3Small, "perf":0},
    "ResNet101": {"model":tf.keras.applications.ResNet101, "perf":0},
    "ResNet101V2": {"model":tf.keras.applications.ResNet101V2, "perf":0},
    "ResNet152": {"model":tf.keras.applications.ResNet152, "perf":0},
    "ResNet152V2": {"model":tf.keras.applications.ResNet152V2, "perf":0},
    "ResNet50": {"model":tf.keras.applications.ResNet50, "perf":0},
    "ResNet50V2": {"model":tf.keras.applications.ResNet50V2, "perf":0},
    "VGG16": {"model":tf.keras.applications.VGG16, "perf":0},
    "VGG19": {"model":tf.keras.applications.VGG19, "perf":0},
    "Xception": {"model":tf.keras.applications.Xception, "perf":0}
}

# Create the generators
train_datagen,val_datagen,train_generator,val_generator=create_gen()
print('\n')

# Fit the models
for name, model in models.items():
    
    # Get the model
    m = get_model(model['model'])
    models[name]['model'] = m
    
    start = perf_counter()
    
    # Fit the model
    history = m.fit(train_generator,
    steps_per_epoch=train_generator.n//train_generator.batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.n//val_generator.batch_size,
    epochs=10,
    verbose=2)
    
    # Sav the duration and the val_accuracy
    duration = perf_counter() - start
    duration = round(duration,2)
    models[name]['perf'] = duration
    print(f"{name:20} trained in {duration} sec")
    
    val_acc = history.history['val_accuracy']
    models[name]['val_acc'] = [round(v,4) for v in val_acc]

Found 1691 images belonging to 2 classes.
Found 345 images belonging to 2 classes.


29089792/29084464 [==============================] - 0s 0us/step
Epoch 1/10
26/26 - 772s - loss: 0.7153 - accuracy: 0.7277 - val_loss: 0.3236 - val_accuracy: 0.8719
Epoch 2/10
26/26 - 98s - loss: 0.3587 - accuracy: 0.8384 - val_loss: 0.2600 - val_accuracy: 0.8938
Epoch 3/10
26/26 - 96s - loss: 0.2831 - accuracy: 0.8728 - val_loss: 0.2479 - val_accuracy: 0.8906
Epoch 4/10
26/26 - 96s - loss: 0.2708 - accuracy: 0.8881 - val_loss: 0.2706 - val_accuracy: 0.8938
Epoch 5/10
26/26 - 96s - loss: 0.2505 - accuracy: 0.9017 - val_loss: 0.2342 - val_accuracy: 0.9062
Epoch 6/10
26/26 - 96s - loss: 0.2391 - accuracy: 0.9121 - val_loss: 0.2636 - val_accuracy: 0.9125
Epoch 7/10
26/26 - 96s - loss: 0.2310 - accuracy: 0.9201 - val_loss: 0.2284 - val_accuracy: 0.9125
Epoch 8/10
26/26 - 96s - loss: 0.2215 - accuracy: 0.9103 - val_loss: 0.3780 - val_accuracy: 0.8719
Epoch 9/10
26/26 - 95s - loss: 0.2226 - accuracy: 0.9127 